This code is to perform some time series analysis on the data 

In [17]:
import pandas as pd
from datetime import datetime,timedelta
import yfinance as yf
eps_df = pd.read_csv('eps_data.csv')
#code to get avg stock price from yfinance for the ticker in each end date and save it to a new column
yf.Screener()



In [27]:
#eps_df contains eps values for a given time period for each ticker
#this code runs through each line and generates a unified time series data set for each ticker
dates = pd.date_range(eps_df['start'].min(), eps_df['end'].max(), freq='d')
#print(len(dates))
df = pd.DataFrame(index=dates)
for i, row in eps_df.iterrows():
    date_range = pd.date_range(start=row['start'], end=row['end'])
    # limit to quarterly earnings only
    if len(date_range) >85 and len(date_range) < 95:
        df.loc[date_range, row['ticker']] = row['EarningsPerShareDiluted']
df.to_csv('eps_by_date.csv')


/var/folders/zl/jy5jnx1j7w12x8wpwfsn9n6m0000gq/T/ipykernel_4030/2172179954.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[date_range, row['ticker']] = row['EarningsPerShareDiluted']
/var/folders/zl/jy5jnx1j7w12x8wpwfsn9n6m0000gq/T/ipykernel_4030/2172179954.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[date_range, row['ticker']] = row['EarningsPerShareDiluted']
/var/folders/zl/jy5jnx1j7w12x8wpwfsn9n6m0000gq/T/ipykernel_4030/2172179954.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is u

In [30]:
#Checking for missing values. sucessful
# Assuming new_df is your DataFrame
missing_values = {}
new_df = df.copy()
for column in new_df.columns:
    # Drop leading and trailing NaNs
    trimmed_col = new_df[column].dropna()
    first_valid_index = trimmed_col.first_valid_index()
    last_valid_index = trimmed_col.last_valid_index()
    
    # Slice the column to exclude leading and trailing NaNs
    if first_valid_index is not None and last_valid_index is not None:
        trimmed_col = trimmed_col.loc[first_valid_index:last_valid_index + timedelta(days=1)]
    
    # Count the number of missing values in the trimmed column
    missing_count = trimmed_col.isna().sum()
    missing_values[column] = missing_count

# Print the number of missing values for each column
for ticker, count in missing_values.items():
    if count > 0:
        print(f"{ticker}: {count} missing values")